In [30]:
!pip install pretty_midi

In [31]:
import os
import pretty_midi
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [32]:
midi_folder = "/content/midi"

In [33]:
notes=[]


In [34]:
for file in os.listdir(midi_folder):
  if file.endswith(".mid"):
    midi = pretty_midi.PrettyMIDI(os.path.join(midi_folder, file))
    for instrument in midi.instruments:
      if not instrument.is_drum:
        for note in instrument.notes:
          notes.append(note.pitch)
print("Total notes collected:", len(notes))


Total notes collected: 1255


In [35]:
encoder = LabelEncoder()
encoded_notes = encoder.fit_transform(notes)


In [36]:
seq_length = 50
sequences = []
targets = []


In [37]:
for i in range(len(encoded_notes) - seq_length):
  sequences.append(encoded_notes[i:i+seq_length])
  targets.append(encoded_notes[i+seq_length])

In [38]:
X = np.array(sequences)
y = np.array(targets)


In [39]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=len(encoder.classes_))

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [41]:
model = Sequential([Embedding(input_dim=len(encoder.classes_),output_dim=100,  input_length=seq_length),
LSTM(256,return_sequences=True),
LSTM(256),
Dense(len(encoder.classes_),activation='softmax') ])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [42]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()
model.fit(X, y, batch_size=64, epochs=20)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 16s 577ms/step - loss: 3.2616
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 564ms/step - loss: 2.6050
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 552ms/step - loss: 2.2783
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 539ms/step - loss: 2.0392
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 508ms/step - loss: 1.9132
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 562ms/step - loss: 1.7821
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 602ms/step - loss: 1.7109
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 579ms/step - loss: 1.5395
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 565ms/step - loss: 1.3776
Epoch 10/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 561ms/step - loss: 1.3802
Epoch 11/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 517ms/step - loss: 1.3014
Epoch 12/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 511ms/step - loss: 1.6716
Epoch 13/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 565ms/step - loss: 1.3958
Epoch 14/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 555ms/step - loss: 1.2653
Epoch 15/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 

In [43]:
import random

In [44]:
seed_idx = random.randint(0, len(X)-1)
seed = X[seed_idx]


In [50]:
generated = []
input_seq = seed.copy()

In [51]:
for _ in range(200): # Generate 200 notes
  pred = model.predict(input_seq[np.newaxis, :], verbose=0)
  next_note = np.argmax(pred)
  generated.append(next_note)
  input_seq = np.append(input_seq[1:], next_note)


In [67]:
def notes_to_midi(note_ids, encoder,output_path="generated_output.mid"):
  midi = pretty_midi.PrettyMIDI()
  inst =  pretty_midi.Instrument(program=0)
  time = 0

  for note_id in note_ids:
    pitch = encoder.inverse_transform([note_id])[0]
    note = pretty_midi.Note(velocity=100, pitch=pitch, start=time, end=time + 0.5)
    inst.notes.append(note)
    time += 0.5
  midi.instruments.append(inst)
  midi.write(output_path)


notes_to_midi(generated, encoder)

In [71]:
from google.colab import files
files.download("generated_output.mid")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>